# Analysis of POC validation exercise results

In [13]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score

In [14]:
columndict={'Anti-Tax Avoidance Directive (ATAD)':'Anti-Tax Avoidance Directive (ATAD)', 
            'Controlled Foreign Corporation':'Controlled Foreign Corporation',
            'Corporate Tax':'Corporate Tax', 
            'Country-by-country reporting':'Country-by-Country Reporting', 
            'Digital Tax':'Digital Tax',
            'Double Tax Treaty ':'Double Tax Treaty', 
            'Economic substance':'Economic substance',
            'EU Mandatory Disclosure Directive (DAC6)':'EU Mandatory Disclosure Directive (DAC6)', 
            'GAAR ':'General anti-avoidance rule (GAAR)', 
            'GST':'Goods and services tax (GST)',
            'Interest Deductibility ':'Interest deductibility', 
            'Local File':'Local file', 
            'Management of a Tax Function':'(Management of a) Tax function',
            'Mandatory disclosure rules':'Mandatory disclosure rules', 
            'Master File':'Master file', 
            'Multilateral Instrument':'Multilateral Instrument (MLI)',
            'Mutual Agreement Procedure':'Mutual agreement procedure', 
            'Permanent Establishment':'Permanent Establishment (PE)',
            'Principal purpose test':'Principal purpose test', 
            'Section 892':'Section 892', 
            'Sovereign Immunity':'Sovereign Immunity',
            'Tax audit':'Tax audit', 
            'Tax compliance':'Tax compliance', 
            'Tax dispute':'Tax dispute',
            'Tax Governance Framework':'Tax Governance Framework', 
            'VAT':'Value-added tax (VAT)', 
            'Withholding Tax':'Withholding Tax'}
def imbibe_validation(filepath):
    def func(item):
        if(item=='Y'):
            return 1
        else: return 0
    df_v1=pd.read_excel(filepath, skiprows=4, nrows=27).T
    categories=df_v1.loc['Category']
    df_v1=pd.DataFrame(df_v1.drop(index='Category').values, columns=categories)
    df_v1=df_v1.applymap(func)
    df_v1.rename(columns=columndict, inplace=True)
    return df_v1

def imbibe_country_validation(filepath):
    countries_list=['Australia','Brazil','China','France','India','Japan','South Korea','Spain','UK','USA']
    def func(item):
        if(item=='Y'):
            return 1
        else: return 0
    df_v1=pd.read_excel(filepath, skiprows=33, nrows=10).T
    countries=df_v1.loc['Country']
    df_v1=pd.DataFrame(df_v1.drop(index='Country').values, columns=countries)
    df_v1=df_v1.applymap(func)
#     df_v1.columns = countries_list
    return df_v1

In [15]:
def make_predictions(df_prob, cutoff):
    df_pred_cut=df_prob.copy()
    df_pred_cut[df_pred_cut.columns]=(df_prob.values>cutoff)*1
    return df_pred_cut

def mean_score(scorer, df_probabilities, df_v1, df_v2, df_v3, cutoff, **kwargs):
    total=0
    total+=scorer(df_v1, make_predictions(df_probabilities, cutoff), **kwargs)
    total+=scorer(df_v2, make_predictions(df_probabilities, cutoff), **kwargs)
    total+=scorer(df_v3, make_predictions(df_probabilities, cutoff), **kwargs)
    return total/3

def mean_analyst_score(scorer, df_v1, df_v2, df_v3, **kwargs):
    p1s=p2s=[df_v1, df_v2, df_v3]
    total=0
    count=0
    for ii, p1 in enumerate(p1s):
        for jj, p2 in enumerate(p2s):
            if ii==jj:
                pass
            else:
                total+=scorer(p1, p2, **kwargs)
                count+=1
    return total/count

def cat_by_cat(valid, predict, topics):
    df_results=pd.DataFrame({'f1':f1_score(valid, predict, average=None), 
                             'precision':precision_score(valid, predict, average=None), 
                             'recall':recall_score(valid, predict, average=None),
                             'occurences':valid.apply(sum)
                            })
    display(df_results)
    
def read_subcat_files(a1_path, a2_path, a3_path, prob_path):
    df_v1=imbibe_validation(a1_path)
    df_v2=imbibe_validation(a2_path)
    df_v3=imbibe_validation(a3_path)
    df_probabilities=pd.read_csv(prob_path)
    try:
        df_probabilities.drop(columns=['Unnamed: 0'], inplace=True)
    except: pass
    df_probabilities=df_probabilities[df_v1.columns]
    return df_v1, df_v2, df_v3, df_probabilities

def read_country_files(a1_path, a2_path, a3_path, country_path):
    df_v1=imbibe_country_validation(a1_path)
    df_v2=imbibe_country_validation(a2_path)
    df_v3=imbibe_country_validation(a3_path)
    df_countries=pd.read_csv(country_path, index_col=0)
    return df_v1, df_v2, df_v3, df_countries

def print_hamm_f1_prec_rec(y_true, y_pred, title):
    print('{} hamming: {:.4f}, F1: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'
          .format(title, hamming_loss(y_true, y_pred), f1_score(y_true, y_pred, average='weighted'), 
                  precision_score(y_true, y_pred, average='weighted'), recall_score(y_true, y_pred, average='weighted') ))
    return f1_score(y_true, y_pred, average='weighted')

def scores_at_each_cutoff(start, end, step, df_v1, df_v2, df_v3, df_probabilities):
    for cutoff in np.arange(start, end, step):
        print('cutoff: {:.1f}, Hamming: {:.4f},  F1: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, hits/doc:{:.2f}'.format(
            cutoff,
            mean_score(hamming_loss, df_probabilities, df_v1, df_v2, df_v3, cutoff), 
            mean_score(f1_score, df_probabilities, df_v1, df_v2, df_v3, cutoff, average='weighted'), 
            mean_score(precision_score, df_probabilities, df_v1, df_v2, df_v3, cutoff, average='weighted'), 
            mean_score(recall_score, df_probabilities, df_v1, df_v2, df_v3, cutoff, average='weighted'), 
            np.sum(make_predictions(df_probabilities, cutoff).apply(sum))/20 ))

def label_comparison(df_v1, df_v2, df_v3, df_pred_cut, main_model_cutoff):
    for df, agent in zip([df_v1, df_v2, df_v3, df_pred_cut], ['analyst 1', 'analyst 2', 'analyst 3', 'computer @ '+str(main_model_cutoff)]):
        print('Number of labels tagged by {}: {}'.format(agent, np.sum(df.apply(sum))))
        
def analyst_summary(df_v1, df_v2, df_v3):
    print('\n', 'Intra-analyst scores. Hamming: {:.4f},  F1: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'.format(
        mean_analyst_score(hamming_loss, df_v1, df_v2, df_v3), 
        mean_analyst_score(f1_score, df_v1, df_v2, df_v3, average='weighted'), 
        mean_analyst_score(precision_score, df_v1, df_v2, df_v3, average='weighted'), 
        mean_analyst_score(recall_score, df_v1, df_v2, df_v3, average='weighted') ))
    
def analyze_subcat_results(a1_path, a2_path, a3_path, prob_path, main_model_cutoff=0.3, verbose=True):
    '''Takes in analyst probabilities and returns summary statistics on superset scores.'''
    
    df_v1, df_v2, df_v3, df_probabilities=read_subcat_files(a1_path, a2_path, a3_path, prob_path)
    
    df_pred_cut=make_predictions(df_probabilities, main_model_cutoff)
    #Potentially load fine-tuned predictions also
    
    if verbose:
        scores_at_each_cutoff(0.1, 1.0, 0.1, df_v1, df_v2, df_v3, df_probabilities)
        label_comparison(df_v1, df_v2, df_v3, df_pred_cut, main_model_cutoff)
        analyst_summary(df_v1, df_v2, df_v3)
        print('\n')
        
    for ii in range(1,4):
        df_vote=(df_v1+df_v2+df_v3)//ii
        if verbose:
            print('Total number of categories identified', np.sum(df_vote.apply(sum)))
        title='Majority of '+ str(ii) 
        if(ii==1):
            score=print_hamm_f1_prec_rec(df_vote>0, df_pred_cut, title)
        elif(verbose):
            print_hamm_f1_prec_rec(df_vote>0, df_pred_cut, title)
    if verbose:
        print('\n')
        print('Superset breakdown @ cut-off {}'.format(main_model_cutoff))
        cat_by_cat(((df_v1+df_v2+df_v3)//1)>0, df_pred_cut, df_v1.columns)
    return score
    
def analyze_country_results(a1_path, a2_path, a3_path, country_path, verbose=True):
    '''Takes in analyst probabilities and returns summary statistics on superset scores.'''
    
    df_v1, df_v2, df_v3, df_countries=read_country_files(a1_path, a2_path, a3_path, country_path)
    if verbose:
        analyst_summary(df_v1, df_v2, df_v3)
        print('\n')
        
    for ii in range(1,4):
        df_vote=(df_v1+df_v2+df_v3)//ii
        if verbose:
            print('Total number of countries identified', np.sum(df_vote.apply(sum)))
        title='Majority of '+ str(ii)
        if(ii==1):
            score=print_hamm_f1_prec_rec(df_vote>0, df_countries, title)
        elif(verbose):
            print_hamm_f1_prec_rec(df_vote>0, df_countries, title)
        
    if verbose:
        print('\n')
        print('Superset breakdown')
        cat_by_cat(((df_v1+df_v2+df_v3)//1)>0, df_countries, df_v1.columns)
    return score

In [16]:
def analyze_all_results(a1_path, a2_path, a3_path, prob_path, country_path, main_model_cutoff=0.3, verbose=True):
    subcat_score=analyze_subcat_results(a1_path, a2_path, a3_path, prob_path, main_model_cutoff=main_model_cutoff, verbose=verbose)
    country_score=analyze_country_results(a1_path, a2_path, a3_path, country_path, verbose=verbose)
    return subcat_score, country_score

# Generate ensembles

In [17]:
def generate_ensemble(bertpath, v1path, outpath):
    df_bert=pd.read_csv(bertpath)
    df_v1=pd.read_csv(v1path)
    df_v1.drop(columns=['Unnamed: 0'], inplace=True)
    df_mixed=(df_bert+df_v1)/2
    df_mixed.to_csv(outpath), index=False)
    
generate_ensemble('./data/test1_BERT_results.csv', './data/test1_probabilities.csv', './data/test1_bert_modelv1.csv')
generate_ensemble('./data/test2_BERT_results.csv', './data/test2_probabilities.csv', './data/test2_bert_modelv1.csv')
generate_ensemble('./data/test3_BERT_results.csv', './data/test3_probabilities.csv', './data/test3_bert_modelv1.csv')

# Results below
## Overall results

In [18]:
import os
def concat_per_pax(ex1dir, ex2dir, ex3dir):
    df_ex1=imbibe_validation(ex1dir)
    df_ex2=imbibe_validation(ex2dir)
    df_ex3=imbibe_validation(ex3dir=)
    df_full=pd.concat([pd.concat([df_ex1, df_ex2], axis=0), df_ex3], axis=0)
    return df_full

def concat_ctry_per_pax(ex1dir, ex2dir, ex3dir):
    df_ex1=imbibe_country_validation(ex1dir)
    df_ex2=imbibe_country_validation(ex2dir)
    df_ex3=imbibe_country_validation(ex3dir)
    df_full=pd.concat([pd.concat([df_ex1, df_ex2], axis=0), df_ex3], axis=0)
    return df_full

def superset_analyst_score(scorer, df_v1, df_v2, df_v3, **kwargs):
    p1s=[df_v1, df_v2, df_v3]
    p2s=[((df_v2+df_v3)//1)>1, ((df_v1+df_v3)//1)>1, ((df_v2+df_v1)//1)>1 ]
    total=0
    count=0
    for p1, p2 in zip(p1s, p2s):
        print(scorer, scorer(p2, p1, **kwargs))
        total+=scorer(p2, p1, **kwargs)
        count+=1
    return total/count

def analyst_superset_summary(df_v1, df_v2, df_v3):
    print('\n', 'Intra-analyst scores. Hamming: {:.4f},  F1: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'.format(
        superset_analyst_score(hamming_loss, df_v1, df_v2, df_v3), 
        superset_analyst_score(f1_score, df_v1, df_v2, df_v3, average='weighted'), 
        superset_analyst_score(precision_score, df_v1, df_v2, df_v3, average='weighted'), 
        superset_analyst_score(recall_score, df_v1, df_v2, df_v3, average='weighted') ))    

def individual_superset_analyst_score(scorer, df_v1, df_v2, df_v3, df_model, **kwargs):
    p1s=[df_v1, df_v2, df_v3]
    p2s=[((df_v2+df_v3)//1)>1, ((df_v1+df_v3)//1)>1, ((df_v2+df_v1)//1)>1 ]
    analyst_scores=[]
    model_scores=[]
    for p1, p2 in zip(p1s, p2s):
        analyst_scores.append(list(scorer(p2, p1, **kwargs)))
        model_scores.append(list(scorer(p2, df_model, **kwargs)))
    return [item for onelist in analyst_scores for item in onelist ], [item for onelist in model_scores for item in onelist]
    
def results_combiner(ex1a1, ex2a1, ex3a1, ex1a2, ex2a2, ex3a2, ex1a3, ex2a3, ex3a3, 
                     cattest1, cattest2, cattest3, ctrytest1, ctrytest2, ctrytest3, main_model_cutoff):
    df_analyst_1=concat_per_pax(ex1a1, ex2a1, ex3a1)
    df_analyst_2=concat_per_pax(ex1a2, ex2a2, ex3a2)
    df_analyst_3=concat_per_pax(ex1a3, ex2a3, ex3a3)
#     df_analyst_1.to_csv('./combined_analyst1.csv')  
    df_probabilities=pd.concat([pd.concat([pd.read_csv(cattest1)[df_analyst_1.columns], 
                                           pd.read_csv(cattest2)[df_analyst_1.columns]], axis=0), 
                                pd.read_csv(cattest3)[df_analyst_1.columns]], axis=0)
    df_pred_cut=make_predictions(df_probabilities, main_model_cutoff)
    analyst_summary(df_analyst_1, df_analyst_2, df_analyst_3)
    df_vote=(df_analyst_1+df_analyst_2+df_analyst_3)//1
    analyst_superset_summary(df_analyst_1, df_analyst_2, df_analyst_3)
    print('Total number of categories identified per doc', np.sum(df_vote.apply(sum))/60, np.sum(df_pred_cut.apply(sum))/60)
    title='Superset score ' 
    p2s=[((df_analyst_2+df_analyst_3)//1)>1, ((df_analyst_1+df_analyst_3)//1)>1, ((df_analyst_2+df_analyst_1)//1)>1 ]
    score=0
    for p2 in p2s:
        score+=print_hamm_f1_prec_rec(p2, df_pred_cut, title)
    print('mean score', score/3)
    score=print_hamm_f1_prec_rec(df_vote>0, df_pred_cut, title)

    print('\n')
    print('Superset breakdown @ cut-off {}'.format(main_model_cutoff))
    cat_by_cat(df_vote>0, df_pred_cut, df_analyst_1.columns)
    
#     analyst_scores, model_scores=individual_superset_analyst_score(f1_score, df_analyst_1, df_analyst_2, df_analyst_3, df_pred_cut, average=None)
#     return analyst_scores, model_scores

    '''This section for countries'''
    df_analyst_1=concat_ctry_per_pax(ex1dir, ex2dir, ex3dir, 0)
    df_analyst_2=concat_ctry_per_pax(ex1dir, ex2dir, ex3dir, 1)
    df_analyst_3=concat_ctry_per_pax(ex1dir, ex2dir, ex3dir, 2)

    df_countries=pd.concat([pd.concat([pd.read_csv(ctrytest1, index_col=0), 
                                           pd.read_csv(ctrytest2, index_col=0)], axis=0), 
                                pd.read_csv(ctrytest3, index_col=0)], axis=0)
    analyst_summary(df_analyst_1, df_analyst_2, df_analyst_3)
    df_vote=((df_analyst_1+df_analyst_2+df_analyst_3)//1)>0
    analyst_superset_summary(df_analyst_1, df_analyst_2, df_analyst_3)
    print('Total number of countries identified per doc', np.sum(df_vote.apply(sum))/60, np.sum(df_countries.apply(sum))/60)
    title='Superset score ' 
    score=print_hamm_f1_prec_rec(df_vote, df_countries, title)
    print('\n')
    print('Superset breakdown')
    cat_by_cat(df_vote, df_countries, df_analyst_1.columns)    
    
results_combiner('./data/POC1_DarrenYang.xlsm', './data/POC2_DarrenYang.xlsm', './data/POC3_DarrenYang.xlsm',
                 './data/POC1_KirstinGallagher.xlsm', './data/POC2_KirstinGallagher.xlsm', './data/POC3_KirstinGallagher.xlsm',
                 './data/POC1_RichardMadden.xlsm', './data/POC2_RichardMadden.xlsm', './data/POC3_RichardMadden.xlsm',
                 './data/test1_bert_modelv1.csv', './data/test2_bert_modelv1.csv', './data/test3_bert_modelv1.csv', 
                 './data/test1_country_predictions.csv', './data/test2_country_predictions.csv', 
                 './data/test3_country_predictions.csv', 0.6)



 Intra-analyst scores. Hamming: 0.1226,  F1: 0.6351, Precision: 0.7516, Recall: 0.6761
<function hamming_loss at 0x000000F0379E1E18> 0.0734567901234568
<function hamming_loss at 0x000000F0379E1E18> 0.16913580246913582
<function hamming_loss at 0x000000F0379E1E18> 0.06604938271604938
<function f1_score at 0x000000F0379E19D8> 0.6940646552370409
<function f1_score at 0x000000F0379E19D8> 0.5622198964619266
<function f1_score at 0x000000F0379E19D8> 0.6973807586734708
<function precision_score at 0x000000F0379E1BF8> 0.7579834054834055
<function precision_score at 0x000000F0379E1BF8> 0.420365328072095
<function precision_score at 0x000000F0379E1BF8> 0.7034492974572558
<function recall_score at 0x000000F0379E1C80> 0.6944444444444444
<function recall_score at 0x000000F0379E1C80> 0.9398496240601504
<function recall_score at 0x000000F0379E1C80> 0.7668711656441718

 Intra-analyst scores. Hamming: 0.1029,  F1: 0.6512, Precision: 0.6273, Recall: 0.8004
Total number of categories identified per doc 

,f1,precision,recall,occurences
Category,,,,
Anti-Tax Avoidance Directive (ATAD),0.800000,1.000000,0.666667,3
Controlled Foreign Corporation,0.285714,1.000000,0.166667,12
Corporate Tax,0.571429,0.952381,0.408163,49
Country-by-Country Reporting,0.428571,1.000000,0.272727,11
Digital Tax,0.628571,1.000000,0.458333,24
Double Tax Treaty,0.697674,0.789474,0.625000,24
Economic substance,0.600000,0.562500,0.642857,14
EU Mandatory Disclosure Directive (DAC6),0.857143,1.000000,0.750000,4
General anti-avoidance rule (GAAR),0.823529,0.777778,0.875000,8



 Intra-analyst scores. Hamming: 0.0622,  F1: 0.8233, Precision: 0.8476, Recall: 0.8336
<function hamming_loss at 0x000000F0379E1E18> 0.058333333333333334
<function hamming_loss at 0x000000F0379E1E18> 0.08
<function hamming_loss at 0x000000F0379E1E18> 0.04833333333333333
<function f1_score at 0x000000F0379E19D8> 0.8159108590410648
<function f1_score at 0x000000F0379E19D8> 0.769666490820337
<function f1_score at 0x000000F0379E19D8> 0.8488010038386763
<function precision_score at 0x000000F0379E1BF8> 0.8507382073708604
<function precision_score at 0x000000F0379E1BF8> 0.6302554387961627
<function precision_score at 0x000000F0379E1BF8> 0.8076265389876882
<function recall_score at 0x000000F0379E1C80> 0.7959183673469388
<function recall_score at 0x000000F0379E1C80> 1.0
<function recall_score at 0x000000F0379E1C80> 0.9069767441860465

 Intra-analyst scores. Hamming: 0.0622,  F1: 0.8115, Precision: 0.7629, Recall: 0.9010
Total number of countries identified per doc 2.2333333333333334 1.6
Supers

,f1,precision,recall,occurences
Country,,,,
Australia,0.769231,0.909091,0.666667,15
Brazil,1.000000,1.000000,1.000000,8
China,0.900000,0.900000,0.900000,10
France,0.666667,0.818182,0.562500,16
India,0.937500,1.000000,0.882353,17
Japan,0.823529,0.777778,0.875000,8
South Korea,0.545455,0.750000,0.428571,7
Spain,0.800000,1.000000,0.666667,9
UK,0.620690,1.000000,0.450000,20
